In [200]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Stat models
from statsmodels.tsa.vector_ar.var_model import VAR

## Handle processing sampled data


In [201]:
# Function to read all csv files in a directory and return a dictionary of dataframes
def read_csv_files(directory: str) -> dict:
    dataframes = {}
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            dataframes[filename.replace('.csv', '')] = df
    return dataframes

In [202]:
# Retrieve a list of DataFrames
dataframes = read_csv_files("data")

In [203]:
# Convert each column in every DataFrame to lowercase
for key in dataframes:
    dataframes[key].columns = map(str.lower, dataframes[key].columns)

Converting the `date` column to datetime object, this process has to be handled individually since each DataFrame has a different date format. We cannot let Pandas infers the date format for each DataFrame since it can be prone to infer the wrong format. Below are the format of each DataFrame:
- Philippines: MM/DD/YYYY
- Singapore: MM/DD/YYYY
- India: YYYY-MM-DD
- United Kingdom: DD/MM/YYYY
- Mexico: YYYY-MM-DD
- Japan: YYYY-MM-DD
- Vietnam: DD/MM/YYYY
- Korea: YYYY-MM-DD
- Thailand: YYYY-MM-DD
- Brazil: YYYY-MM-DD
- Malaysia: DD/MM/YYYY
- Switzerland: YYYY-MM-DD
- China: DD/MM/YYYY
- Russia: YYYY-MM-DD
- United States: YYYY-MM-DD

In [204]:
date_format_mapping = {
  'philippines': '%m/%d/%Y',
  'singapore': '%m/%d/%Y',
  'india': '%Y-%m-%d',
  'uk': '%d/%m/%Y',
  'mexico': '%Y-%m-%d',
  'japan': '%Y-%m-%d',
  'vietnam': '%d/%m/%Y',
  'korea': '%Y-%m-%d',
  'thailand': '%Y-%m-%d',
  'brazil': '%Y-%m-%d',
  'malaysia': '%d/%m/%Y',
  'switzerland': '%Y-%m-%d',
  'china': '%d/%m/%Y',
  'russia': '%Y-%m-%d',
  'us': '%Y-%m-%d',
}

# Convert the date columns to datetime objects
for key in dataframes:
    try:
      dataframes[key]['date'] = pd.to_datetime(
        dataframes[key]['date'], 
        format=date_format_mapping[key]
      )
    except Exception as e:
      print(f"Error occurred for country: {key}")
      print(f"Error message: {str(e)}")

In [205]:
# Sort the dataframes by date in ascending order
for key in dataframes:
    dataframes[key] = dataframes[key].sort_values(by='date')

In [206]:
# Reset the index of the dataframes
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

In [207]:
# Extract only open, high, low, close columns
for key in dataframes:
    dataframes[key] = dataframes[key][['date', 'open', 'high', 'low', 'close']]

In [208]:
for df in dataframes.values():
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3055 entries, 0 to 3054
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3055 non-null   datetime64[ns]
 1   open    3055 non-null   object        
 2   high    3055 non-null   object        
 3   low     3055 non-null   object        
 4   close   3055 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 119.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3181 entries, 0 to 3180
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3181 non-null   datetime64[ns]
 1   open    3181 non-null   object        
 2   high    3181 non-null   object        
 3   low     3181 non-null   object        
 4   close   3181 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 124.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Ra

Since the columns open, high, low, close have different dtypes (`float64` and `object`) for different DataFrame, they should be converted to `float64`.

In [211]:
# Convert open, high, low, close columns to float
for key in dataframes:
  for col in ['open', 'high', 'low', 'close']:
    if dataframes[key][col].dtype == 'object':
      dataframes[key][col] = dataframes[key][col].str.replace(',', '').astype(float)


In [213]:
# Check whether the columns have been converted to float
for df in dataframes.values():
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3055 entries, 0 to 3054
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3055 non-null   datetime64[ns]
 1   open    3055 non-null   float64       
 2   high    3055 non-null   float64       
 3   low     3055 non-null   float64       
 4   close   3055 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 119.5 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3181 entries, 0 to 3180
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3181 non-null   datetime64[ns]
 1   open    3181 non-null   float64       
 2   high    3181 non-null   float64       
 3   low     3181 non-null   float64       
 4   close   3181 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 124.4 KB
None
<class 'pandas.core.frame.DataFrame'>
Ra

## Data Preprocessing


##### Data imputation

This section aims to handle the missing values present in each DataFrame. Since these stock dataset will eventually be fed into a Vector Auto-regressive (VAR) model, it is quite important to choose a data imputation method that will preserve the temporal and cross-sectional relationships among countries.

Simpler methods such as linear interpolation and forward filling will be implemented on initial experiments, and more complex methods like Kalman filter will be considered if the imputed data are not sufficient for VAR models.

In [214]:
"""
Function to check for missing values in the dataframes
"""
def check_missing_values(dataframes: dict) -> None:
    for key in dataframes:
        print(f"Missing values for {key}:")
        print(dataframes[key].isnull().sum()) 
        print("\n")

In [215]:
check_missing_values(dataframes)

Missing values for philippines:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for singapore:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for india:
date      0
open     39
high     39
low      39
close    39
dtype: int64


Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date      0
open     49
high     49
low      49
close    49
dtype: int64


Missing values for japan:
date      0
open     93
high     93
low      93
close    93
dtype: int64


Missing values for vietnam:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date      0
open     66
high     66
low      66
close    66
dtype: int64


Missing values for thailand:
date      0
open     73
high     73
low      73
close    73
dtype: int64


Missing values for brazil:
date       0
open     462
high     462
low      462
close    462
dtype:

In [216]:
# Impute missing values in the dataframes using linear interpolation method
for key in dataframes:
    dataframes[key] = dataframes[key].interpolate(method='linear')

In [218]:
# Check whether the missing values have been imputed
check_missing_values(dataframes)

Missing values for philippines:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for singapore:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for india:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for vietnam:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for brazil:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for malaysi

##### Volatility calculation

In [219]:
"""
Function to calculate volatility
"""
def calculate_volatility(dataframes: dict) -> dict:
    for key in dataframes:
        df = dataframes[key]

        high = df['high']
        low = df['low']
        close = df['close']
        open = df['open']

        # Calculate volatility
        df['volatility'] = 0.551 * (high - low)**2 - 0.019 * (
            (close - open) * (high + low - 2*open) - 2 * (high - open) * (low - open)
        ) - 0.383 * (close - open)**2
    return dataframes

In [220]:
dataframes = calculate_volatility(dataframes)

In [221]:
# Check whether the volatility column has been added
for df in dataframes.values():
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3055 entries, 0 to 3054
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        3055 non-null   datetime64[ns]
 1   open        3055 non-null   float64       
 2   high        3055 non-null   float64       
 3   low         3055 non-null   float64       
 4   close       3055 non-null   float64       
 5   volatility  3055 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 143.3 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3181 entries, 0 to 3180
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        3181 non-null   datetime64[ns]
 1   open        3181 non-null   float64       
 2   high        3181 non-null   float64       
 3   low         3181 non-null   float64       
 4   close       3181 non-null   float64       
 5

#### Window extraction

In this section, I will perform window extraction for the following time period:
- **Window 1:** 02.01.2002 to 17.09.2007
- **Window 2:** 18.09.2007 to 27.10.2011 (India's dataset starts from 18.09.2007)
- **Window 3:** 28.10.2011 to 31.12.2018 (Philippines's dataset starts from 28.10.2011)
- **Window 4:** 02.01.2019 to 31.12.2022 
- **Window 5:** 02.01.2023 to 30.04.2024 

In [ ]:
"""
Function to check whether a dataframe has data populated within a specified date range
"""

def check_date_range(dataframes: pd.DataFrame, start_date: str, end_date: str) -> dict:
    # Create datetime objects for the start and end of the year range
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Dictionary to hold the result
    result = {}
    
    # Iterate through each DataFrame
    for name, df in dataframes.items():
        # Check if the DataFrame contains any dates within the specified year range
        contains_data = df['date'].between(start_date, end_date).any()
        
        # Update the result dictionary
        result[name] = contains_data
    
    return result

"""
Function to extract each DataFrame's data within a specified date range
"""

def extract_date_range(dataframes: pd.DataFrame, start_date: str, end_date: str) -> dict:
    # Create datetime objects for the start and end of the year range
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Dictionary to hold the result
    result = {}
    
    # Iterate through each DataFrame
    for name, df in dataframes.items():
        # Extract the data within the specified year range
        extracted_data = df[df['date'].between(start_date, end_date)]
        
        # Update the result dictionary
        result[name] = extracted_data
    
    return result

##### 02.01.2002 - 17.09.2007: Recovery period from Dot-com Bubble

In [ ]:
# Check whether each DataFrame contains data from 02.01.2002 to 17.09.2007
window_1_start_date = '2002-01-02'
window_1_end_date = '2007-09-17'
check_date_range(dataframes, window_1_start_date, window_1_end_date)

The following countries do not have data available for this time period:
1. Philippine
2. Singapore
3. India
4. Vietnam
5. Brazil
6. Malaysia
7. Russia

These 7 countries will be be considered in VAR experiment for this time period, with the addition of China since it only has data from 2005.

In [ ]:
dfs_window_1 = extract_date_range(dataframes, window_1_start_date, window_1_end_date)

# Check whether each DataFrame contains data from 02.01.2002 to 17.09.2007
dfs_window_1

In [ ]:
# Remove countries that do not have data for the specified date range (02.01.2002 to 17.09.2007) from the dictionary
for key in list(dfs_window_1.keys()):
  if dfs_window_1[key].empty:
    del dfs_window_1[key]

# Remove the 'china' DataFrame from the dictionary
del dfs_window_1['china']

In [ ]:
len(dfs_window_1)

##### 18.09.2007 - 27.10.2011: Global Financial Crisis

In [ ]:
# Check whether each DataFrame contains data from 18.09.2007 to 27.10.2011
window_2_start_date = '2007-09-18'
window_2_end_date = '2011-10-27'
check_date_range(dataframes, window_2_start_date, window_2_end_date)

Most countries have data from 18.09.2007 to 27.10.2011, with the exception of Philippines and Russia. In addition, there are Singapore, which only has data from 07.03.2011. These 3 countries will be remove from this time window.

In [ ]:
dfs_window_2 = extract_date_range(dataframes, window_2_start_date, window_2_end_date)

# Check whether each DataFrame contains data from 18.09.2007 to 27.10.2011
dfs_window_2

In [ ]:
# Remove countries that do not have data for the specified date range (02.01.2002 to 17.09.2007) from the dictionary
for key in list(dfs_window_2.keys()):
  if dfs_window_2[key].empty:
    del dfs_window_2[key]

# Remove the 'singapore' DataFrame from the dictionary
del dfs_window_2['singapore']

In [ ]:
len(dfs_window_2)

##### 28.10.2011 - 31.12.2018: Recovery from Global Financial Crisis

In [ ]:
# Check whether each DataFrame contains data from 28.10.2011 to 31.12.2018
window_3_start_date = '2011-10-28'
window_3_end_date = '2018-12-31'
check_date_range(dataframes, window_3_start_date, window_3_end_date)

Most DataFrame appears to have data from 28.10.2011 to 31.12.2018, except from Russia, which only has data from 2013. Russia will be removed from this time window.

In [ ]:
dfs_window_3 = extract_date_range(dataframes, window_3_start_date, window_3_end_date)

# Check whether each DataFrame contains data from 28.10.2011 - 31.12.2018
dfs_window_3

In [ ]:
del dfs_window_3['russia']

In [ ]:
len(dfs_window_3)

##### 02.01.2019 - 31.12.2022: COVID-19 pandemic

In [ ]:
# Check whether each DataFrame contains data from 02.01.2019 to 31.12.2022
window_4_start_date = '2019-01-02'
window_4_end_date = '2022-12-31'
check_date_range(dataframes, window_4_start_date, window_4_end_date)

In [ ]:
dfs_window_4 = extract_date_range(dataframes, window_4_start_date, window_4_end_date)

# Check whether each DataFrame contains data from 02.01.2019 to 31.12.2022
dfs_window_4

##### 02.01.2023 - 30.04.2024: Recovery from COVID-19 pandemic

In [ ]:
# Check whether each DataFrame contains data from 02.01.2023 to 30.04.2024
window_5_start_date = '2023-01-02'
window_5_end_date = '2024-04-30'
check_date_range(dataframes, window_5_start_date, window_5_end_date)

In [ ]:
dfs_window_5 = extract_date_range(dataframes, window_5_start_date, window_5_end_date)

# Check whether each DataFrame contains data from 02.01.2023 to 30.04.2024
dfs_window_5